# Exercise 2: Naive Bayes text classifier

### Training the model

  - Calculate $P(y)$ for each class label in the training data
  - Calculate $P(x_i|y)$ for each feature (term) for each class label in the training data 
  
$$P(x_i|y)=\frac{c_{i,y} + 1}{c_i + m}$$

where 
  - $c_{i,y}$ is the number of times term $x_i$ appears in class $y$
  - $c_i$ is the total number of times term $x_i$ appears in the collection
  - $m$ is the number of classes


### Applying the model

Return the class $y \in Y$ that maximizes $P(y) \prod_{x_i} P(x_i|y)$.

Note that we need to consider $x_i$ at each *word position* in the document. Thus, we need to multiply with $P(x_i|y)$ as many times as $x_i$ appears in the document.
We can rewrite it as: $$P(y|x) \propto P(y) \prod_{i \in d} P(x_i|y)^{c_{i,d}}$$ where $c_{i,d}$ is the number of times term $i$ appears in document $d$.

Finally, we perform the computations in the log domain, that is, $$\log P(y) +  \sum_{i=1}^n (c_{i,d} \cdot\log P(x_i|y))$$

## 1) Probability estimations

The estimation of probabilities $P(x_i|y)$ and $P(y)$ are refactored to a separate class to make them testable.

In [1]:
class NBProbabilityEstimator:
    
    def get_prior_prob(self, y, num_classes):
        """Computes the class prior probability, P(y)."""
        return 1 / num_classes
    
    def get_term_prob(self, count_inclass, count_total, num_unique_terms):
        """Computes the smoothed term probability for a given class, P(x_i|y).
        
        Args:
          count_inclass: Number of times the term appears in the given class.
          count_total: Number of times the term appears in the collection.
          num_unique_terms: Size of the vocabulary.
        Returns:
          The probability P(x_i|y).
        """
        return (count_inclass + 1) / (count_total + num_unique_terms)

### Unit testing for `NBProbabilityEstimator`

In [2]:
import unittest

class TestNBProbabilityEstimator(unittest.TestCase):

    def test_prior_prob(self):
        nbpe = NBProbabilityEstimator()
        self.assertEqual(nbpe.get_prior_prob(1, 4), 0.25)

    def test_term_prob(self):
        nbpe = NBProbabilityEstimator()
        self.assertEqual(nbpe.get_term_prob(5, 20, 10), 0.2)
        self.assertEqual(nbpe.get_term_prob(74, 90, 10), 0.75)
        self.assertEqual(nbpe.get_term_prob(0, 6, 10), 0.0625)
        

unittest.main(argv=[''], verbosity=2, exit=False)

test_prior_prob (__main__.TestNBProbabilityEstimator) ... ok
test_term_prob (__main__.TestNBProbabilityEstimator) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


## 2) Naive Bayes classifier

Implement training and prediction for a Naive Bayes classifier.  We are operating with dense matrices for simplicity.

In [3]:
import numpy as np
import math

class NBClassifier:

    def __init__(self):
        self._nbprob = NBProbabilityEstimator()
        self._num_classes = 0
        self._term_prob = None
        
    
    def fit(self, X, y):
        """Fit the model.
        
        Args:
          X: Document-term matrix where rows correspond to documents and columns correspond to terms.
          y: Class labels corresponding to documents.
        """        
        self._num_classes = len(np.unique(y))
        num_docs = len(X)
        num_terms = len(X[0])        
        self._term_prob = np.zeros((num_terms, self._num_classes))
        
        # Iterating through the vocabulary
        for i in range(num_terms):
            # Holds c_{i,j} values, i.e., the number of times term i appears with class j.
            class_count = [0] * self._num_classes
            for d in range(num_docs):
                class_count[y[d]] += X[d, i]
                        
            # Calculate P(x_i|y)
            total_count = sum(class_count)
            for j in range(self._num_classes):
                self._term_prob[i, j] = self._nbprob.get_term_prob(class_count[j], total_count, num_terms)

                
    def _predict_instance(self, x):
        """Predict class for a single instance (document).
        
        Args:
          x: Term vector.
        Returns:
          The predicted class label (0-indexed).
        """
        probs = []
        
        for j in range(self._num_classes):
            p = math.log(self._nbprob.get_prior_prob(j, self._num_classes))
            for i in range(len(x)):
                if x[i] > 0:
                    p += x[i] * math.log(self._term_prob[i, j])
            probs.append(p)
            
        # Get the class with the highest probability.
        return probs.index(max(probs))
        
    
    def predict(self, X):
        """Make predictions for a set of documents.
        
        Args:
          X: Document-term matrix.
        Returns:
          Array with predictions.
        """
        predictions = []
        # Iterate through test documents.
        for x in X:
            predictions.append(self._predict_instance(x))
        return np.asarray(predictions)
        

## 3) Testing on real data

We will be using a subset of the 20Newsgroups collection.

In [4]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

Get term frequencies using `CountVectorizer`. (We ignore terms that appear in less than 10 documents to speed up computation.)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df=10)
X_train_counts = count_vect.fit_transform(train.data)
X_test_counts = count_vect.transform(test.data)

Train and apply the model. Note that we convert sparse matrices to dense ones. This is not efficient and should be avoided when working with large datasets. Nevertheless, this simplifies the implementation for this exercise.

In [ ]:
nb = NBClassifier()
nb.fit(X_train_counts.toarray(), train.target)
pred = nb.predict(X_test_counts.toarray())

Evaluation.

In [ ]:
from sklearn import metrics

metrics.accuracy_score(test.target, pred)

## Optional exercise

If you're done, try to implement it without making a conversion to dense matrices.

Also, do we really need to precompute and store all term probabilities?

## Feedback

Please give (anonymous) feedback on this exercise by filling out [this form]().